<img src="https://drive.google.com/uc?id=1-cL5eOpEsbuIEkvwW2KnpXC12-PAbamr" style="Width:1000px">

# Hyperparameters for SVMs

The purpose of this exercise is to become familiar with support verctor classifier, and exploring the different classes of algorithms we can use. Support vector machine algorithms can be very computationally intensive, especially when using non-linear kernels. Our dataset today is modest, but it does have 100'000 instances. This dataset size requires us to be mindful about what algorithm we deploy.

We will also use this as a good excuse to learn and practice hyperparameter tuning!

👇 Consider the following dataset.

In [ ]:
from nbta.utils import download_data
download_data(id='1cbL-nzeziC8SgATbxroSITfh5ukbucWW')

In [ ]:
import pandas as pd

data = pd.read_csv("raw_data/data.csv")

data.head()

Each observations represents the characterstic of a wine, and the last column is the quality rating [from 0 being the poorest to 10 being the best].

The dataset is large (about 100'000 instances) but has missing values. The goal is to build a recommender system that will classify an unknown wine quality as either 'high' or 'low'. And we will do this in a pipeline.

# Data preparation
As you can see, right now you have 10 classes representing the wine quality. We need to turn this into a binary problem. Do the following:
* We will consider wines with a quality rating <6 as being of 'poor' quality, and above or equal to 6 as being of 'good' quality
* Use the <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html'>`.apply()` function in Pandas</a> and a <a href='https://towardsdatascience.com/apply-and-lambda-usage-in-pandas-b13a1ea037f7'>`lambda function`</a> 
* Replace the values in the `quality rating` column by 0 if <6 or 1 if >=1

In [ ]:
data['quality rating'] = data['quality rating'].apply(lambda x: 0 if x<6 else 1)

## Strategy

As stated in the introduction, we will need to be efficient and clever about what we do given the size of the dataset. For instance, we might not want to do cross validation on our entire dataset since it is very large, and SVMs can be slow. Also, choosing what *falvor* of SVM Classifier we use is important. What we will do is the following:

1. train/test split the whole dataset with 20% test split (you can reduce the test split slize as your dataset becomes larger)
2. Sample a small portion of the train set (**10% in our case**) as the data as our `validation_set`. Note that this will only be used to speed up some of our tests: we will use cross validation for hyperparameter searching. This means that **you should not replace the `X_train` and `y_train`** that you made in step 1. Instead, you can use train-test-split on the `train_set` but assign the 90% of the train set to the variable `_` (underscore). This is a convention in Python for a variable we won't be using.
3. We will test the speed of different versions of `SVMs` using the `validation_set` (10% of your `train_set`)
4. We will then do hyperparameter searches on the fastest algorithm using the original `train_set`
5. Finally, we can train the final version of our algorithm on our `train_set` and test it on our `test_set`.

<span style="color:blue">**Ready? Let's do it!**</span>

## Preparing the data

To save you some time, the code for the data preparation is already given to you. I want you to focus more on the SVMs and on hyperparameter tuning in this exercuse. We will do the following:
- Create a target (`y`) containing the `data['quality rating']`
- Create a feature (`X`) vector containing all columns except the `data['quality rating']` 
- Train/test split the data (80%/20%)
- Further train-test-split the `train_split` into a `validation_split` (10%) and `_` (dummy variable)
- You should end up with these: `X_train`, `y_train`, `X_test`, `y_test`, `X_val`, `y_val` (and a `_` variable)


Then, create a simple pipeline to do the following:
- Impute missing values using a `SimpleImputer`
- Scale the features using a `StandardScaler` scaler

Onces you have your pipeline fitted, use it to transform the `X_train`, `X_test`, and `X_val` into variables `X_train_prep`, `X_test_prep`, and `X_val_prep` (we will need the original variables later in the notebook).

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

# Testing different SVMs training speed

***Aims:*** Our aim here is to see if there are any differences in how fast the different versions of **SVMs** work. Note that we will limit ourselves to *<span style="color:teal">linear SVMs</span>, i.e. no other kernels.

You will test the following classes of SVM: `SVC(kernel=linear)`, `LinearSVC(dual=True)`, and `SGDClassifier(loss="hinge")`. For each of the algorithm, do the following:

1. Instanciate a classifier without thinkering the hyperparameters in a separate notebook cell
2. Create a new cell, and in that cell, add the following line of code first: `%%timeit`. This will ensure that the code will be run multiple times, and the average time of the code cell will be return. This will allow you to know how fast/slow it is to train each algorithm.
3. Now `.fit()` your algorithm using `X_val_prep` and `y_val`

record the timing of each of your experiment, and save the name of the algorithm as a string (`svc`,`linearsvc`,`sgd`) in a variable called `fast_algorithm`


In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

<details>
    <summary>Explanation</summary>

- The `SVC()` algorithm supports kernels (including the non-linear kernels), but it comes at a cost: under the hood, it uses a library known as `libsvm`, which uses a **one-vs-one** strategy when fitting multi-class classifiers (slower)
- The `LinearSVC` is based on `liblinear` and uses a **one-vs-all** approach and penalises the intercept. **It does not support kernels**. But it is much faster then `SVC()` and in fact the `dual=False` version is even a bit faster than `sgd` for our dataset (but I asked you to test `dual=True`).
- The `SGDClassifier` with `hinge` loss also does **not support kernels** but uses stockastic gradient descent to solve for the loss, which is very fast (**SGD** is what we will use for deep-learning)
    
As you can see, the choice of which algorithm to use should be part of your strategy. If you need to use **kernels**, you will need to stick with `SVC()`. For very large datasets, `SGDClassifier(loss="hinge")` is your best bet.
    
</details>

### ☑️ Check your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('svms_speed',
                         fast_algorithm = fast_algorithm
)

result.write()
print(result.check())

# Finding the best parameters and the best model

Now create your `RandomizedSearchCV` object (let's call it `rand_search`). Remember to fill the 'params' dictionary, and use this guideline for that:
* A `penalty` term between either [`l1` , `l2`]
* A range of value of `alpha` between 0.000001 and 10 is reasonable; Explore the `scipy.stats.uniform` class and it's documentation in order to generate a uniform distribution for your values of `alpha`.
* `max_iter` values of [1000, 5000, 10000].
* Use `accuracy` as your scoring metric, and a value of `cv=5`
* In a separate cell, run 60 models with your `RandomizedSearchCV` (`fit` to `X_train_prep` and `y_train`), and don't forget to use all of the CPUs on your machine to speed up training (`n_jobs=-1`)

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

#### Take note of how long this took


# Finding hyperparameters for pipeline

We can also use `RandomizedSearchCV` object (let's call it `rand_search_prep`) in pipelines! This is much more convenient as you no longer need to pre-transform your features, and you can also check the impact of your data preparation decisions. <br>
Do the following:
1. Combine your `data preparation` pipeline with an `SGDClassifier(loss='hinge' )` classifier into one pipline.
2. Then, gridsearch the following parameters:<br>
**For the classifier (as before):**
* A `penalty` term between either [`l1` , `l2`]
* A range of value of `alpha` between 0.000001 and 10 is reasonable; Explore the `scipy.stats.uniform` class and it's documentation in order to generate a uniform distribution for your values of `alpha`.
* `max_iter` values of [1000, 5000, 10000].
* Use `accuracy` as your scoring metric, and a value of `cv=5`
* In a separate cell, run 60 models with your `RandomizedSearchCV` (`fit` to your original `X_train` and `y_train`), and don't forget to use all of the CPUs on your machine to speed up training (`n_jobs=-1`)<br>
**For the data pipeline:**
* Check whether a `StandardScaler()` or a `RobustScaler()` is best!

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

## Best hyperparameters and best estimator

Use the `best_params_` attribute of you `rand_search_prep` to discover what the best hyperparameters were. Then, create a variable (`rs_model`) and assign to it the value of your `best_estmator_`. The `best_estmator_` returns a trained model, so this is now your best `SVM` trained model.

Finally, using your `rs_model` calculate the `accuracy_score` of the `test_set` and save this value into a variable named `rs_score`

In [ ]:
# ADD YOUR CODE HERE -- You can create new markdown and code cells
                    
                    
                    

### ☑️ Check your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('score',
                         score = rs_score
)

result.write()
print(result.check())

# 🏁 Finished!

Well done! <span style="color:teal">**Push your exercise to GitHub**</span>, and move on to the next one.